<p style="font-weight:bold;"> <span style="font-size: 36px"> Test Map Template Export and Import </span> </p>

Uncomment this row if you are debugging this test or running it separately. 

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
var workspace = Workspace.CreateNew();
workspace.InitializeFrom(DataSource);

# Test Empty Database

In [0]:
(await workspace.Query<DataNode>().ToArrayAsync()).Length.Should().NotBe(0);

In [0]:
(await workspace.Query<Portfolio>().ToArrayAsync()).Length.Should().NotBe(0);

In [0]:
(await workspace.Query<InsurancePortfolio>().ToArrayAsync()).Length.Should().NotBe(0);

In [0]:
(await workspace.Query<ReinsurancePortfolio>().ToArrayAsync()).Length.Should().NotBe(0);

In [0]:
(await workspace.Query<GroupOfContract>().ToArrayAsync()).Length.Should().NotBe(0);

In [0]:
(await workspace.Query<GroupOfInsuranceContract>().ToArrayAsync()).Length.Should().NotBe(0);

In [0]:
(await workspace.Query<GroupOfReinsuranceContract>().ToArrayAsync()).Length.Should().NotBe(0);

In [0]:
(await workspace.Query<DataNodeState>().ToArrayAsync()).Length.Should().NotBe(0);

In [0]:
(await workspace.Query<DataNodeParameter>().ToArrayAsync()).Length.Should().NotBe(0);

# Test Data Node Parameter

In [0]:
var singleParam = (await workspace.Query<SingleDataNodeParameter>().Where(x => x.DataNode == "DT1.1").ToArrayAsync()).Single();

In [0]:
singleParam.CashFlowPeriodicity.Should().Be((CashFlowPeriodicity)default);

In [0]:
singleParam.InterpolationMethod.Should().Be((InterpolationMethod)default);

# Test Map Template

## Data Node

In [0]:
//Workspace.InitializeFrom(DataSource);

static var partition = new PartitionByReportingNode() { ReportingNode = "CH" };
static var filename = "TestMapTemplateDataNode";

//Workspace.InitializeFrom(DataSource);
await workspace.Partition.SetAsync<PartitionByReportingNode>( partition );

var ips = workspace.Query<InsurancePortfolio>().ToArray();
var rps = workspace.Query<ReinsurancePortfolio>().ToArray();
var gics = workspace.Query<GroupOfInsuranceContract>().ToArray();
var grics = workspace.Query<GroupOfReinsuranceContract>().ToArray();

var exportResult = await Export.ToExcel(filename)
    .WithSource(workspace)
    .PortfolioConfiguration<ReinsurancePortfolio>()
    .PortfolioConfiguration<InsurancePortfolio>()
    .GroupofContractConfiguration<GroupOfReinsuranceContract>(typeof(ReinsurancePortfolio))
    .GroupofContractConfiguration<GroupOfInsuranceContract>(typeof(InsurancePortfolio))
    .MainTabConfiguration(partition)
    .WithActivityLog()
    .ExecuteAsync();

exportResult.ActivityLog.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
await Export.ToExcel(filename)
    .WithSource(workspace)
    .PortfolioConfiguration<ReinsurancePortfolio>()
    .PortfolioConfiguration<InsurancePortfolio>()
    .GroupofContractConfiguration<GroupOfReinsuranceContract>(typeof(ReinsurancePortfolio))
    .GroupofContractConfiguration<GroupOfInsuranceContract>(typeof(InsurancePortfolio))
    .MainTabConfiguration(partition)
    .WithActivityLog()
    .ExecuteAsync()

In [0]:
// Run this only after having imported the file exported in the above cell.
/*
// check that exporter generates only the desired columns 
var extension = ".xlsx";
var stream = await Project.FileStorage.ReadAsync(filename + extension);
var tables = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;
var mainCols = new[]{nameof(PartitionByReportingNode.ReportingNode)};
var portfolioCols = new[]{nameof(Portfolio.SystemName),nameof(Portfolio.DisplayName),nameof(Portfolio.ContractualCurrency),
                          nameof(Portfolio.LineOfBusiness),nameof(Portfolio.OciType),nameof(Portfolio.ValuationApproach)};
var gicCols = new[]{nameof(GroupOfContract.SystemName),nameof(GroupOfContract.DisplayName),nameof(GroupOfContract.AnnualCohort),
                    nameof(GroupOfContract.LiabilityType),nameof(InsurancePortfolio),nameof(GroupOfContract.Profitability)};
var gricCols = new[]{nameof(GroupOfContract.SystemName),nameof(GroupOfContract.DisplayName),nameof(GroupOfContract.AnnualCohort),
                     nameof(GroupOfContract.LiabilityType),nameof(ReinsurancePortfolio),nameof(GroupOfContract.Profitability),nameof(GroupOfContract.Partner)};

mainCols.Intersect(tables.Tables[Main].Columns.Select(x => x.ColumnName).ToArray()).Count().Should().Be(mainCols.Length);
portfolioCols.Intersect(tables.Tables[nameof(InsurancePortfolio)].Columns.Select(x => x.ColumnName).ToArray()).Count().Should().Be(portfolioCols.Length); 
portfolioCols.Intersect(tables.Tables[nameof(ReinsurancePortfolio)].Columns.Select(x => x.ColumnName).ToArray()).Count().Should().Be(portfolioCols.Length); 
gicCols.Intersect(tables.Tables[nameof(GroupOfInsuranceContract)].Columns.Select(x => x.ColumnName).ToArray()).Count().Should().Be(gicCols.Length); 
gricCols.Intersect(tables.Tables[nameof(GroupOfReinsuranceContract)].Columns.Select(x => x.ColumnName).ToArray()).Count().Should().Be(gricCols.Length); 

// check that imported data matches the exported data
await Import.FromFile(filename+".xlsx").WithFormat(ImportFormats.DataNode).WithTarget(workspace).ExecuteAsync();
Utils.EqualityComparer<InsurancePortfolio>(ips, workspace.Query<InsurancePortfolio>().ToArray());
Utils.EqualityComparer<ReinsurancePortfolio>(rps, workspace.Query<ReinsurancePortfolio>().ToArray());
Utils.EqualityComparer<GroupOfInsuranceContract>(gics, workspace.Query<GroupOfInsuranceContract>().ToArray());
Utils.EqualityComparer<GroupOfReinsuranceContract>(grics, workspace.Query<GroupOfReinsuranceContract>().ToArray());
*/

In [0]:
workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

## Data Node State

In [0]:
static var partition = new PartitionByReportingNodeAndPeriod() { ReportingNode = "CH", Year = 2021, Month = 3 };
static var filename = "TestMapTemplateDataNodeState";

workspace.InitializeFrom(DataSource);
await workspace.Partition.SetAsync<PartitionByReportingNode>( new PartitionByReportingNode() { ReportingNode = partition.ReportingNode } );
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>( partition );

var dataNodeStates = await workspace.Query<DataNodeState>()
                        .Where(x => (x.Year == partition.Year && x.Month <= partition.Month) || x.Year < partition.Year)
                        .GroupBy(x => x.DataNode)
                        .Select(x => x.OrderByDescending(y => y.Year).ThenByDescending(y => y.Month))
                        .Select(x => x.Last())
                        .ToArrayAsync();

var exportResult = await Export.ToExcel(filename)
    .WithSource(workspace)
    .StateEnumConfiguration() 
    .DataNodeStateConfiguration(dataNodeStates)
    .MainTabConfiguration(partition)
    .WithActivityLog()
    .ExecuteAsync();

exportResult.ActivityLog.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
await Export.ToExcel(filename)
    .WithSource(workspace)
    .StateEnumConfiguration() 
    .DataNodeStateConfiguration(dataNodeStates)
    .MainTabConfiguration(partition)
    .WithActivityLog()
    .ExecuteAsync()

In [0]:
// Run this only after having imported the file exported in the above cell.
/*
// check that exporter generates only the desired columns 
var extension = ".xlsx";
var stream = await Project.FileStorage.ReadAsync(filename + extension);
var tables = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;
var mainCols = new[]{nameof(PartitionByReportingNodeAndPeriod.ReportingNode),
                     nameof(PartitionByReportingNodeAndPeriod.Year),
                     nameof(PartitionByReportingNodeAndPeriod.Month)};
var dataNodeStateCols = new[]{nameof(DataNodeState.DataNode),nameof(DataNodeState.State)};
mainCols.Intersect(tables.Tables[Main].Columns.Select(x => x.ColumnName).ToArray()).Count().Should().Be(mainCols.Length);
dataNodeStateCols.Intersect(tables.Tables["DataNodeState"].Columns.Select(x => x.ColumnName).ToArray()).Count().Should().Be(dataNodeStateCols.Length);

// check that imported data matches the exported data 
var dataNodeStates = workspace.Query<DataNodeState>().ToArray();
await Import.FromFile(filename+".xlsx").WithFormat(ImportFormats.DataNodeState).WithTarget(Workspace).ExecuteAsync();
// Workspace is empty because ValidateDataNodeStatesAsync removes the entry, since this is already present in the DataSource.
(workspace.Query<DataNodeState>().ToArray().Except(dataNodeStates)).Should().BeEmpty();
*/

In [0]:
workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

## DataNodeParameter

In [0]:
workspace.InitializeFrom(DataSource);
static var partition = new PartitionByReportingNodeAndPeriod() { ReportingNode = "CH", Year = 2020, Month = 12 };
var partitionByReportingNode = (await workspace.Query<PartitionByReportingNode>().Where(x => x.ReportingNode == partition.ReportingNode).ToArrayAsync()).Single();
static var filename = "TestMapTemplateDataNodeParameter";
var singleDataNodeParamBm = await workspace.Query<SingleDataNodeParameter>().Where(x => x.Year == partition.Year && x.Month == partition.Month && x.Partition == partitionByReportingNode.Id ).ToArrayAsync();
var interDataNodeParamBm = await workspace.Query<InterDataNodeParameter>().Where(x => x.Year == partition.Year && x.Month == partition.Month && x.Partition == partitionByReportingNode.Id ).ToArrayAsync();
await workspace.Partition.SetAsync<PartitionByReportingNode>( new PartitionByReportingNode() { ReportingNode = partition.ReportingNode } );
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>( partition );
//--------------------Export Map Template--------------------------
//--------------------From MapTemplate NB--------------------------
var dataNodeParameters = (await workspace.Query<DataNodeParameter>()
                        .Where(x => (x.Year == partition.Year && x.Month <= partition.Month) || x.Year < partition.Year).ToArrayAsync())
                        .GroupBy(x => x.GetType().Name)
                        .ToDictionary(x => x.Key, 
                                      x => x.GroupBy(y => y.DataNode)
                                            .Select(y => y.OrderByDescending(z => z.Year).ThenByDescending(z => z.Month))
                                            .Select(y => y.First())
                                            .ToArray() );
var exportResult = await Export.ToExcel(filename)
    .WithSource(workspace)
    .DataNodeParameterConfiguration(dataNodeParameters)
    .MainTabConfiguration(partition)
    .WithActivityLog()
    .ExecuteAsync();

exportResult.ActivityLog.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
await Export.ToExcel(filename)
    .WithSource(workspace)
    .DataNodeParameterConfiguration(dataNodeParameters)
    .MainTabConfiguration(partition)
    .WithActivityLog()
    .ExecuteAsync()

In [0]:
// Run this only after having imported the file exported in the above cell.
/*
// check that exporter generates only the desired columns 
var extension = ".xlsx";
var stream = await Project.FileStorage.ReadAsync(filename + extension);
var tables = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;
var mainCols = new[]{nameof(PartitionByReportingNodeAndPeriod.ReportingNode),
                     nameof(PartitionByReportingNodeAndPeriod.Year),
                     nameof(PartitionByReportingNodeAndPeriod.Month),
                     nameof(PartitionByReportingNodeAndPeriod.Scenario)};
var singleDataNodeParamCols = new[]{nameof(DataNode),
                                    nameof(SingleDataNodeParameter.PremiumAllocation),
                                    nameof(SingleDataNodeParameter.CashFlowPeriodicity),
                                    nameof(SingleDataNodeParameter.InterpolationMethod),
                                    };
var interDataNodeParamCols = new[]{nameof(DataNodeParameter.DataNode),
                                   nameof(InterDataNodeParameter.LinkedDataNode),
                                   nameof(InterDataNodeParameter.ReinsuranceCoverage)};
mainCols.Intersect(tables.Tables[Main].Columns.Select(x => x.ColumnName).ToArray()).Count().Should().Be(mainCols.Length);
singleDataNodeParamCols.Intersect(tables.Tables[nameof(SingleDataNodeParameter)].Columns.Select(x => x.ColumnName).ToArray()).Count().Should().Be(singleDataNodeParamCols.Length);
interDataNodeParamCols.Intersect(tables.Tables[nameof(InterDataNodeParameter)].Columns.Select(x => x.ColumnName).ToArray()).Count().Should().Be(interDataNodeParamCols.Length);
// check that imported data matches the exported data 
await Import.FromFile(filename+".xlsx").WithFormat(ImportFormats.DataNodeParameter).WithTarget(workspace).ExecuteAsync();
var expectedSingleDataNodeParamBm = workspace.Query<SingleDataNodeParameter>().Where(x => x.Year == partition.Year && x.Month == partition.Month).ToArray();
var expectedInterDataNodeParamBm = workspace.Query<InterDataNodeParameter>().ToArray();
Utils.EqualityComparer<SingleDataNodeParameter>(singleDataNodeParamBm, expectedSingleDataNodeParamBm);
Utils.EqualityComparer<InterDataNodeParameter>(interDataNodeParamBm, expectedInterDataNodeParamBm);
*/

In [0]:
workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

# Test Import Validation

## Data Node State: entries cannot be imported which change status from Inactive to Active

In [0]:
workspace.InitializeFrom(DataSource);
var partition = workspace.Query<PartitionByReportingNode>().Where(x => x.ReportingNode == "CH").ToArray().FirstOrDefault();
await workspace.Partition.SetAsync<PartitionByReportingNode>(partition);

var itemInactive = new DataNodeState { Year = 2022, Month = 9, DataNode = "GIC1", State = State.Inactive, Partition = partition.Id };
var itemActive = new DataNodeState { Year = 2022, Month = 9, DataNode = "GIC1", State = State.Active, Partition = partition.Id };

// Define GIC1 State as Inactive
var persistentDataNodeDataByDataNode = new Dictionary<string, DataNodeData> 
{
    ["GIC1"] = new DataNodeData() {DataNode = "GIC1", State = itemInactive.State, Year = itemInactive.Year, Month =  itemInactive.Month},
};

// Set GIC1 from Inactive into Active
await workspace.DeleteAsync(Workspace.Query<DataNodeState>().ToArray());
await workspace.UpdateAsync(itemActive);

// Test Validation
Activity.Start();

await workspace.ValidateDataNodeStatesAsync(persistentDataNodeDataByDataNode);
Activity.HasErrors().Should().Be(true);
var log = Activity.Finish().Errors.First().ToString().Substring(40);
(log.Substring(0,log.Length-2) == Error.ChangeDataNodeState.GetMessage("GIC1")).Should().Be(true);

In [0]:
workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

## Data Node State: Validation removes the entry when the DataSource already contains a previous version  

In [0]:
workspace.InitializeFrom(DataSource);
var partition = workspace.Query<PartitionByReportingNode>().Where(x => x.ReportingNode == "CH").ToArray().FirstOrDefault();
await workspace.Partition.SetAsync<PartitionByReportingNode>(partition);

var itemPrevious = new DataNodeState { Year = 2022, Month = 3, DataNode = "GIC1", State = State.Active, Partition = partition.Id };
var itemImport = new DataNodeState { Year = 2022, Month = 9, DataNode = "GIC1", State = State.Active, Partition = partition.Id };

// Define persisted Data Node State
var persistentDataNodeDataByDataNode = new Dictionary<string, DataNodeData>
{
    ["GIC1"] = new DataNodeData() {DataNode = itemPrevious.DataNode, State = itemPrevious.State, Year = itemPrevious.Year, Month =  itemPrevious.Month},
};

// Reimport Data Node State for GIC1 with State unchanged
await workspace.DeleteAsync(workspace.Query<DataNodeState>().ToArray());
await workspace.UpdateAsync(itemImport);
await workspace.ValidateDataNodeStatesAsync(persistentDataNodeDataByDataNode);

// Check that the new redundant State is removed from the Workspace
workspace.Query<DataNodeState>().ToArray().Should().BeEmpty();

In [0]:
//Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
workspace.Dispose();